In [26]:
import sys
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from tqdm import tqdm
import time
from selenium import webdriver
import time
import json

In [5]:
def list_to_file(python_list, path):
    with open(path, 'w') as f:
        f.write('\n'.join(python_list))
        
def remove_dupes_file(file_path, new_file_path):
    with open(file_path, 'r') as f:
        lines = f.read().splitlines()
        unique_lines = set(lines)
    with open(new_file_path, 'w') as g:
        g.write('\n'.join(unique_lines))

In [3]:
remove_dupes_file('list_of_presidents.txt', 'unique_list_of_presidents.txt')

In [4]:
presidents_full = open('unique_list_of_presidents.txt').read().split('\n')
presidents_full\

['Ronald Reagan',
 'Bob Dole',
 'Mitt Romney',
 'Al Gore',
 'Spiro Agnew',
 'Hubert Humphrey',
 'Barack Obama',
 'George W. Bush',
 'John F. Kennedy',
 'Dan Quayle',
 'Walter Mondale',
 'Michael Dukakis',
 'Mike Pence',
 'Gerald Ford',
 'Barry Goldwater',
 'John Kerry',
 'Richard Nixon',
 'Nelson Rockefeller',
 'Hillary Clinton',
 'Bill Clinton',
 'George McGovern',
 'Lyndon B. Johnson',
 'Dick Cheney',
 'Joe Biden',
 'Donald Trump',
 'John McCain',
 'Jimmy Carter',
 'George H. W. Bush']

In [6]:
def get_links2(url):
    '''Extracts all links on a URL'''
    soup = BeautifulSoup(url)
    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    return links

In [23]:
def scroll_bottom(url):
    '''Takes URL and returns driver object to be passed to bs4'''
    from selenium.webdriver.chrome.options import Options
    #create chrome session
    CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
    WINDOW_SIZE = "1920,1080"
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
#     chrome_options.binary_location = CHROME_PATH
    
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                              options=chrome_options)
    driver.implicitly_wait(30)
    driver.get(url)

    for i in range(10):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(1)

    page_source = driver.page_source
    driver.quit()
    return page_source

In [21]:
def extract_speeches_links_from_millscenter():
    '''Goes to each president's page and extracts speeches transcript's links from it'''
    from selenium.webdriver.chrome.options import Options
   #manually extracted presidents and presidents' ids 
    president_ids = [39, 43, 8396, 40, 35, 42, 38, 34, 37,41]
    presidents = ['Reagan', 'Obama', 'Trump', 'H.W.Bush', 'Lyndon', 'Bush', 'Carter', 'Kennedy', 'Ford', 'Clinton']
    # URL structure = https://millercenter.org/the-presidency/presidential-speeches?field_president_target_id[43]=43
    president_speeches = {}
    for president_id, president in zip(president_ids, presidents):
        url = 'https://millercenter.org/the-presidency/presidential-speeches?field_president_target_id[{}]={}'.format(
        president_id, president_id)
        page_source = scroll_bottom(url)
        #pass Selenium's page_source to bs4 to extract links - this could be done with Selenium
        links = get_links2(page_source)
        #find links with actual speeches - include '/presidential-speeches/'
        speech_links = [link for link in links if link is not None and '/the-presidency/presidential-speeches/' in link]
        president_speeches[president] = speech_links
    n_sp = sum([len(l) for l in president_speeches.values()])
    print("Total speeches collected: {}".format(n_sp))
    return president_speeches

In [9]:
def process_links_list(link_list):
    '''Takes a list of URLS with speeches and returns the clean speeches (from https://millercenter.org/)'''
    root_url = 'https://millercenter.org/'
    all_speeches = []
    for link in tqdm(link_list):
        url = root_url + link
        print(url)
        expanded_page = click_button_selenium(url)
        html_page = requests.get(expanded_page).text
        soup = BeautifulSoup(html_page)
        paragraphs = soup.find_all('p')
        speech_clean = join_one_speech(paragraphs)
        if speech_clean:
            all_speeches.append(speech_clean)
    time.sleep(3)
    print("Finished list of links. {} speeches saved of {} links processed".format(
                    len(all_speeches), len(link_list)))
    return all_speeches

def join_one_speech(speech_paragraphs):
    '''Extracts and processes the text from a single speech/page'''
    speech_joined = []
    for paragraph in speech_paragraphs:
        clean_paragraph = paragraph.get_text(strip=True).replace('\r\n\t\t\t', '')
        speech_joined.append(clean_paragraph)
    speech_joined = ' '.join(speech_joined)
    return speech_joined

In [10]:
def try_get_metadata(xpath, driver):
    '''Given an absolute xpath, try to extract text and return None if
    element doesn't exist'''
    from selenium.common.exceptions import NoSuchElementException
    try:
        data = driver.find_element_by_xpath(xpath).text
        return data
    except NoSuchElementException as e:
        return None

In [60]:
def try_get_metadata(xpath, driver):
    '''Given an absolute xpath, try to extract text and return None if
    element doesn't exist'''
    from selenium.common.exceptions import NoSuchElementException
    try:
        data = driver.find_element_by_xpath(xpath).text
        return data
    except NoSuchElementException as e:
        return None


def dataframe_single_transcript(url):
    '''Takes an URL from millcenter.org and clicks the "View Transcript" button'''
    from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
    from selenium.webdriver.chrome.options import Options

#     CHROME_PATH = '/usr/bin/google-chrome'
    CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
    WINDOW_SIZE = "1920,1080"
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
#     chrome_options.binary_location = CHROME_PATH
    
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                              options=chrome_options)
    driver.implicitly_wait(30)
    count = 5
    while count:
        try:
            driver.get(url)
            break
        except TimeoutException:
            count = count - 1
        except WebDriverException:
            print("Driver not initiated. Trying again.")
            count = count - 1
    
    count = 0
    while count < 5:
        #click transcript button
        try:
            trascript_button = driver.find_element_by_class_name('expandable-text-trigger')
            trascript_button.click()
            #get transcript text
            inner_transcript = driver.find_elements_by_class_name('transcript-inner')[0]
            all_paragraphs = inner_transcript.find_elements_by_tag_name('p')#find_element_by_xpath('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[2]/div[3]/div/div[2]/div[1]/p').text
            transcript_text = [p.text for p in all_paragraphs]
            transcript_text = '\n'.join(transcript_text)
            break
        except NoSuchElementException as e:
            #get transcript text
            try:
                inner_transcript = driver.find_elements_by_class_name('view-transcript')[0]
                all_paragraphs = inner_transcript.find_elements_by_tag_name('p')#find_element_by_xpath('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[2]/div[3]/div/div[2]/div[1]/p').text
                transcript_text = [p.text for p in all_paragraphs]
                transcript_text = '\n'.join(transcript_text)
                break
            except NoSuchElementException as e:
                count += 1
        except ElementClickInterceptedException as e:
            count += 1
    else:
        trascript_text = "Error: Couldn't get text in URL {}, error {}".format(url, e)
        
    #get other speech info
    summary = try_get_metadata('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[1]/div/div/div[2]/p', driver)
    president = try_get_metadata('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[1]/div/div/p[1]', driver)
    source = try_get_metadata('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[1]/div/div/div[1]/span[2]', driver)
    date = try_get_metadata('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[1]/div/div/p[2]', driver)
    word_count = len(transcript_text.split())
    columns = ['Transcript', 'Summary', 'President', 'Source', 'Date', 'URL', 'Word Count']
    #construct df
    speech_df = pd.DataFrame([[transcript_text, summary, president, source, date, url, word_count]], columns=columns)
    display(speech_df)
    driver.quit()
    return speech_df


In [53]:
def append_speeches(speeches_links_dictionary):
    '''Iterates through the dictionary with presindential speeches links and creates 
    a dataframe with all the transcripts plus metadata'''
    all_df = pd.DataFrame()
    for president, speech_urls_list in speeches_links_dictionary.items():
#         if president in ['Reagan', 'Obama', 'Trump', 'H.W.Bush', 'Lyndon', 'Bush', 'Carter']:
#             continue
        print("President: {}".format(president))
        for i, speech_link in enumerate(speech_urls_list):
            print("Processing speech {} of {}".format(i, len(speech_urls_list)))
            url = 'https://millercenter.org' + speech_link
            temp_df = dataframe_single_transcript(url)
            all_df = all_df.append(temp_df)
            temp_df.to_csv(f'{president}.csv')
    return all_df

In [ ]:
president_speeches = extract_speeches_links_from_millscenter()


In [29]:
with open('millscenter_speeches_links.txt', 'w') as f:
    json.dump(president_speeches,f)

In [61]:
all_df = append_speeches(president_speeches)

President: Reagan
Processing speech 0 of 59


,Transcript,Summary,President,Source,Date,URL,Word Count
0,This is the 34th time I'll speak to you from t...,"In this broadcast from the Oval Office, Presid...",Ronald Reagan,National Archives,"January 11, 1989",https://millercenter.org/the-presidency/presid...,3289


Processing speech 1 of 59


,Transcript,Summary,President,Source,Date,URL,Word Count
0,"Well, thank you very much for that warm welcom...",None,Ronald Reagan,None,"December 16, 1988",https://millercenter.org/the-presidency/presid...,7127


Processing speech 2 of 59


...

,Transcript,Summary,President,Source,Date,URL,Word Count
0,"Thank you very much, thank you.\nMrs. Yandle, ...",President Clinton signs the Family Medical Lea...,Bill Clinton,National Archives,"February 05, 1993",https://millercenter.org/the-presidency/presid...,1045


Processing speech 37 of 39


,Transcript,Summary,President,Source,Date,URL,Word Count
0,"The President. Good afternoon, ladies and gent...",President Bill Clinton addresses the press reg...,Bill Clinton,National Archives,"January 29, 1993",https://millercenter.org/the-presidency/presid...,2327


Processing speech 38 of 39


,Transcript,Summary,President,Source,Date,URL,Word Count
0,"My fellow citizens, today we celebrate the mys...",As the first President elected in the post-Col...,Bill Clinton,National Archives,"January 20, 1993",https://millercenter.org/the-presidency/presid...,1598


In [59]:
%debug

> <ipython-input-57-6b8064c7806d>(54)dataframe_single_transcript()
     52             #get transcript text
     53             try:
---> 54                 inner_transcript = driver.find_elements_by_class_name('transcript-inner')[0]
     55                 all_paragraphs = inner_transcript.find_elements_by_tag_name('p')#find_element_by_xpath('/html/body/div[2]/div/main/div[2]/div[2]/div[2]/article/div/div[2]/div[3]/div/div[2]/div[1]/p').text
     56                 transcript_text = [p.text for p in all_paragraphs]

ipdb> url
'https://millercenter.org/the-presidency/presidential-speeches/november-4-2008-remarks-election-night'
ipdb> q


In [51]:
all_df.President.unique()

array(['Ronald Reagan', 'Barack Obama', 'Donald Trump',
       'George H. W. Bush', 'Lyndon B. Johnson', 'George W. Bush',
       'Jimmy Carter', 'John F. Kennedy', 'Gerald Ford', 'Bill Clinton'],
      dtype=object)

In [ ]:
# combine both temp dfs

In [62]:
all_df.to_csv('millercenter_speeches_df.csv')

In [35]:
tmp = pd.read_csv('temp_speeches.csv', index_col=0)

In [36]:
fil = tmp["President"]!="John F. Kennedy"


In [37]:
tmp = tmp.where(fil) 

In [41]:
tmp = tmp.reset_index()
tmp.drop('index', axis=1, inplace=True)

In [44]:
tmp.drop('level_0', axis=1, inplace=True)

In [47]:
all_speeches = tmp.append([all_df], ignore_index=True)

In [53]:
all_speeches.to_csv('all_millscenter_speeches.csv', index=False)

In [51]:
all_speeches

(225, 5)

In [ ]:
# creating the df: president, year, affiliation, title

In [ ]:
# Don't need bs4 for this website, just selenium - mention in the report